<a href="https://colab.research.google.com/github/Akkuun/M1_IMAGINE_ML_Project/blob/main/ML_Project_Group_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<H1>Projet Machine Learning</H1>
<H2>HAI817I - 2024/2025</H2>
<H3>Classification d'assertions venant d’X (Twitter) selon leur rapport à la science </H3>



<H1>Groupe 8 </H1>
<br>

Duban Mathis: 22102226

Gonzalez Oropeza Gilles : 21602817

Bernardon Vincent : 22009116

LONGLADE Mickaël :22105047




<H2> Sujet </H2>

Ce projet s’inscrit dans le contexte de l’apprentissage supervisé, i.e. les données possèdent des
labels. Il vise à trouver les modèles les plus performants pour prédire si des assertions (une
assertion est une proposition que l'on avance et que l'on soutient comme vraie) faites par des
hommes politiques (par exemple) sont vraies ou fausses.




---



<H1>Tables des matières </H1>



*   Installation





## **Installation**


# Dans cette partie, nous allons installer toutes les librairies que nous allons avoir besoin pour notre projet.




In [93]:
# Installation des librairies pour le projet
!pip install pandas numpy emoji nltk matplotlib seaborn scikit-learn inflect googletrans==4.0.0-rc1

# Importation des différentes librairies utiles pour le notebook

import warnings # Supprime les warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Librairies de manipulation de données (graphique, lecture de données,ect...)
import pandas as pd # Lecture de données
import numpy as np # Array
import seaborn as sns #
import matplotlib.pyplot as plt # Graphique
import sys

# Librairies pour la fonction prepareText
import re # Regular expression
import nltk
from nltk.corpus import stopwords #English stopwords
nltk.download('stopwords') # Téléchargement des stopwords (une seule fois)
import emoji
import inflect # Transformation des chiffres en mots
import re
from googletrans import Translator # Traduction de langues

ERROR: Could not find a version that satisfies the requirement currency-symbol-map (from versions: none)
ERROR: No matching distribution found for currency-symbol-map


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ModuleNotFoundError: No module named 'currency_symbols'

Importation du répertoire de travail sur Google Drive


In [76]:
from google.colab import drive
drive.mount('/content/gdrive/')
path='/content/gdrive/My Drive/M1_IMAGINE_ML/ML_Project/'
sys.path.append(path)
%cd $path
%pwd

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/My Drive/M1_IMAGINE_ML/ML_Project


'/content/gdrive/My Drive/M1_IMAGINE_ML/ML_Project'

Récupération des données du dataSet présent sur le répertoire Google Drive en ligne


In [77]:
# Importation des données
df=pd.read_csv('dataSet/data.csv', sep='\t')
# Lecture des 5 premières lignes pour confirmer la bonne récupération des données
display (df.head())

,Unnamed: 0,tweet_id,text,science_related,scientific_claim,scientific_reference,scientific_context
0,0,316669998137483264,Knees are a bit sore. i guess that's a sign th...,0,0.0,0.0,0.0
1,1,319090866545385472,McDonald's breakfast stop then the gym 🏀💪,0,0.0,0.0,0.0
2,2,322030931022065664,Can any Gynecologist with Cancer Experience ex...,1,1.0,0.0,0.0
3,3,322694830620807168,Couch-lock highs lead to sleeping in the couch...,1,1.0,0.0,0.0
4,4,328524426658328576,Does daily routine help prevent problems with ...,1,1.0,0.0,0.0


## **Ingénierie des données**

Dans cette partie, on s'intérssèe au pré-traitement des données. Afin que chaque éléments de notre base soit utilisable et pertinent on va nettoyer, normaliser et transofmrer nos données afin qu'elle soit préparées et optimale pour nos analyses.


<br>

On va ici répertorier tous les éléments qui doivent être traité :

<br>


| Élément            | Exemple                 | Traitement à effectuer | Exemple après traitement |
|--------------------|------------------------|-------------------------|--------------------------|
| Emojis            | 🏀                     | Suppression des émojis ou remplacement par leur signification | "" / basketball |
| Mention Twitter   | @username               | Remplacement par un Token |  @MENTION |
| Hashtag           | #example                |  Remplacement par un Token | @HASHTAG |
| URL               | http://t.co/XGUfUDoLJB  | Suppression de l'URL | "" |
| Chiffre           | 13                      | Transformation en String | thirdteen |
| Majuscules        | Hello                    | Suppression de la majuscule | hello |
| Ponctuation       | !                        | Suppression de la ponctuation | "" |
| Mots répétés      | cool cool cool         | Normalisation en une seule occurrence | cool |
| Lettres répétées  | that's greeeeeeeat!        | Réduction des répétitions excessives | that's great! |
| Abréviations      | ngl, fr | Remplacement par la version complète | not gonna lie, for real |
| Stopwords (déterminants)         | the, and, a  | Suppression si non pertinent | "" |
| Slang (Argot)     | gonna, dunno, wanna           | Remplacement par des mots standards | going to, do not know, want to |
| Langue étrangère  | bonjour, gracias          | Détection et traduction éventuelle | hello, thank you |
| Caractères spéciaux | §, $, ^                | Suppression des charactères | "" |
| Expressions courantes | btw, lol    | Remplacement par la version complète | by the way, laughing out loud|
| Négation mal formatée | ain't, dunno          | Correction grammaticale | am not, do not know |
| Émojis en Unicode | `\U0001F60D` (😍)      | Conversion en texte lisible | love |
| Symboles de devises |    10\$, 10\€               | Normalisation (ex: "10 euros") | 10 dollars, 10 euros |



<br>

<br> Afin de préparer nos données, on va créer une fonction **prepareText** permettant de préparer toutes nos données




In [95]:
stop_words_set = set(stopwords.words('english'))
translator = Translator() # initialisation du traducteur

# Fonction permettant de préparer la chaîne de charactères passé en paramètre
def prepareText(text):
    #Majuscule, suppression
    data = str(text).lower()  # Convertir en minuscules

    #Emoji (à déterminer)

    #Mention Twitter, transformation en Token
    data = re.sub(r'@\w+', '@MENTION', data)

    #Hashtag, transformation en Token
    data = re.sub(r'#\w+', '@HASHTAG', data)

    #URL, on supprime
    data = re.sub(r'http\S+', '', data)

    #Chiffre, transformation en String
    words = data.split()
    data = ' '.join([inflect.engine().number_to_words(word) if word.isdigit() else word for word in words])

    #Ponctuation, suppression
    data = re.sub(r'[^\w\s]', '', data)

    #Stopwords, suppression
    data = ' '.join([word for word in data.split() if word not in stop_words_set])

    #Charactères spéciaux, suppresion
    data = re.sub(r'[^\w\s]', '', data)

    #Devise, remplacement par sa chaîne de charactere





    #Traduction du tweet
    try:
        data = translator.translate(data, dest='en').text
    except:
        pass

    return data

Exemple du passage de notre fonction


In [96]:
#URL
display("http://t.co/XGUfUDoLJB")
display(prepareText("http://t.co/XGUfUDoLJB"))
print("\n")
#Chiffre
display("3")
display(prepareText("3"))
print("\n")
#Majuscule
display("Hello")
display(prepareText("Hello"))
print("\n")
#Ponctuation
display("Hello!")
display(prepareText("Hello!"))
print("\n")
#Abréviation
display("lol")
display(prepareText("lol"))
print("\n")
#StepWord
#display a exemple sentect with a lot a stepword
display("After planning the project, she carefully researched each step, ensuring the execution was smooth and timely")
display(prepareText("After planning the project, she carefully researched each step, ensuring the execution was smooth and timely"))
print("\n")
#Emojis
display("😍")
display(prepareText("😍"))
print("\n")
#Traductions (dernière étape)
display("我今天去超市买了一些水果")
display(prepareText("我今天去超市买了一些水果"))
print("\n")
#Mention Twitter
display("as @username said it's bad !")
display(prepareText("as @username said it's bad !"))
print("\n")
#Hashtag
display("I went to the theater to see Dune 2 #Dune")
display(prepareText("I went to the theater to see Dune 2 #Dune"))
print("\n")
#Charactères spéciaux
display("§$£")
display(prepareText("§$£"))
print("\n")
#Devices
display("10$ 10£ 10€")
display(prepareText("10$ 10£ 10€"))
print("\n")


'http://t.co/XGUfUDoLJB'

''

'3'

'three'

'Hello'

...

'went theater see dune two HASHTAG'

'§$£'

''

'10$ 10£ 10€'

'10 10 10'

In [80]:
#display all text data
df['text'] = df['text'].apply(prepareText)
